In [1]:
import os,tables,numpy as np,matplotlib.pyplot as plt,pandas as pd,json
from scipy.io import loadmat
from collections import Counter
from dataholder import Data

In [100]:
os.getcwd()

'/media/mhealthra2/Data/heart_sound/Adversarial-Heart-Sound-Classification/codes'

In [104]:
fold_dir = '../../feature/potes_1DCNN/balancedCV/folds/folds_phys_compare_pascal/'
os.listdir(fold_dir)

['compare.mat',
 'fold_a.mat',
 'fold_b.mat',
 'fold_c.mat',
 'fold_d.mat',
 'fold_e.mat',
 'fold_f.mat',
 'pascalA.mat',
 'pascalB.mat',
 'readme.txt']

In [194]:
import matplotlib.pyplot as plt,h5py
from collections import Counter

class Data():
    def __init__(self,path,f,n):
        self.data = h5py.File(path+f, 'r')
        self.file = f
        self.dom = n
        self.trainX = self.data['trainX']
        self.trainY = self.data['trainY'][:][0]
        self.train_parts = self.data['train_parts'][0]
        self.valX = None
        self.valY = None
        self.normfiles, self.abnormfiles = self.parts()
        self.domainY = [n]*self.trainY.shape[0]
        if(f[:3]=='com'):
            self.trainY[self.trainY>0] = 1
            self.valX = self.data['valX']
            self.valY = self.data['valY'][:][0]
            self.valY[self.valY>0] = 1
        elif(f[:3]=='pas'):
            self.trainY[self.trainY<0] = 0
        else:
            self.trainY[self.trainY<0] = 0
        #print(Counter(self.trainY))
        self.normal = Counter(self.trainY)[0]
        self.abnormal = Counter(self.trainY)[1]
        self.total = self.normal+self.abnormal
        if('fold_e' in f):
            nn = ab = 0
            idx = []
            for x in self.trainY:
                if x==0:
                    if(nn<self.abnormal):
                        idx.append(True)
                        nn += 1
                    else:idx.append(False);
                else:
                    if(ab<self.abnormal):
                        idx.append(True)
                        ab += 1
                    else:idx.append(False);
            self.trainY = self.trainY[idx]
            self.trainX = np.transpose(np.transpose(self.trainX)[idx])
            self.train_parts = None
            self.total = nn+ab
            self.domainY = self.domainY[:self.total]
            self.normfiles, self.abnormfiles = nn,ab
    def parts(self):
        y = 0
        nn = 0
        ab = 0
        for x in self.train_parts:
            if(sum(self.trainY[y:y+int(x)])>0):
                ab = ab + 1
            else: nn = nn +1 
            y = int(x)+y
        return nn,ab
    def pie(self):
        colors = ['gold','lightskyblue']
        explode = (0.07, 0)
        size = [self.normal,self.abnormal]
        labels = ['N', 'Ab']
        plt.pie(size,labels=labels,colors = colors,startangle=140,explode=explode,
               shadow=True,autopct=self.value)
        plt.title(self.dom)

In [193]:
Data(fold_dir,'fold_e.mat','e')


57642
5704


In [201]:
class DataMerge():
    def __init__(self):
        self.x_train = self.y_train = self.y_domain = self.train_parts = None
    def merge(self,data):
        if(self.x_train is None):
            self.x_train = data.trainX;
            self.y_train = data.trainY;
            self.y_domain = data.domainY;
            self.train_parts = data.train_parts;
        else:
            self.x_train = np.concatenate((self.x_train,data.trainX),axis = 1)
            self.y_train = np.concatenate((self.y_train,data.trainY),axis = 0)
            self.y_domain = self.y_domain+data.domainY
            #print("before", self.train_parts.shape,data.train_parts.shape)
            if(data.train_parts is not None):
                self.train_parts = np.concatenate((self.train_parts,data.train_parts),axis = 0)            
            #print("AAAAfter", self.train_parts.shape)

In [85]:
def getData(path, folds):
    try:
        with open('../data/domain_filename.json', 'r') as fp:
            foldname = json.load(fp)
    except:
        raise FileNotFoundError("The json file that maps domain character to filename is not here")
        
    allData = DataMerge()
    for c in folds:
        allData.merge(Data(fold_dir,foldname[c],c))
    return allData.x_train, allData.y_train, allData.y_domain, allData.train_parts

In [ ]:
def reshape_folds(x_train, y_train,x_val, y_val):
    x1 = np.reshape(x_train, [x_train.shape[0], x_train.shape[1], 1])
    y_train = np.reshape(y_train, [y_train.shape[0], 1])

    print(x1.shape)
    print(y_train.shape)
    if(x_val is not None):
        v1 = np.transpose(x_val[:, :])

        v1 = np.reshape(v1, [v1.shape[0], v1.shape[1], 1])

        y_val = np.reshape(y_val, [y_val.shape[0], 1])

        print(v1.shape)
        print(y_val.shape)
        return x1, y_train, v1 , y_val
    return x1,y_train

In [202]:
u,i,o,p = getData(fold_dir,'bdefghi')

In [92]:
u.shape,i.shape,len(o),p.shape

((2500, 19734), (19734,), 19734, (839,))

In [10]:
np.unique('abcdcdef')

array(['abcdcdef'], dtype='<U8')

In [98]:
ww = [2,2,2,2]
all([x==2 for x in ww])

True

In [101]:
'fold_e' in "fold_e_dfasfd"

True